In [1]:
#setup; don't mess with this
Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex') #
def buildmonoms(howmany): #important to build the monomials in the right order so our OPAs will be in a reasonable order
    Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex')
    monomslist=[]
    m=0
    for totaldeg in range(0,howmany+1):
        k=0;l=totaldeg;
        while k<=totaldeg and m<=howmany:
            monomslist.append(sage_eval('z1^'+str(l)+'*z2^'+str(k),locals=vars()))#so it's not just strings
            #print (k,l),newcoefs[(k, l)]
            m=m+1
            k=k+1
            l=l-1
    return monomslist
def multideg(monom): #take a monomial in z1 and z2 and return an ordered pair with its multidegree 
                    #(there's probably a way to do this more generally)
    Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex')
    z1deg=monom.degree(z1)
    z2deg=monom.degree(z2)
    return (z1deg,z2deg)
def coeffdict(poly,monomslist): #takes a polynomial (as an expression) and a pregenerated list of monomials and returns a dictionary
                                # keys are ordered pairs of degrees of a monomial, entries are coefficients
    polydict={}
    for monom in monomslist:
        multi=multideg(monom)
        #print(multi)
        d={z1:multi[0],z2:multi[1]}
        #print(d)
        polydict.update({multideg(monom):poly.coefficient(d)})
    return polydict
#inner product
def InnerProd(f,g,monomslist):
    cfsf=coeffdict(f,monomslist)#the coefficients paired into a dictionary with the multi-indices
    cfsg=coeffdict(g,monomslist)
    #print cfsf
    #print cfsg
    pairs=list(set(list(cfsf.keys())+list(cfsg.keys())))
    ip=0
    for alpha in pairs:
        term=cfsf[alpha]*conjugate(cfsg[alpha])
        ip=ip+term
    return ip
#
def buildMatrix(monomslist,fcn,whichpn):
    M=matrix(QQ,whichpn+1,lambda x,y: InnerProd(monomslist[x]*fcn,monomslist[y]*fcn,monomslist))
    return M
def RHSmateq(f,whichpn):
    fcn(z1,z2)=f
    RHSveclist=[conjugate(f(0,0))]
    for n in range(whichpn):
        RHSveclist=RHSveclist+[0]
    return vector(RHSveclist)
# puts the coefficients from a list into the polynomial
def coeffpoly(optimal_list,whichOA,monomslist):
    thepoly='0'
    for coef in range(0,whichOA+1):
            thepoly=thepoly+'+'+str(optimal_list[coef])+'*'+str(monomslist[coef])
            #print (k,l),newcoefs[(k, l)]
    return sage_eval(thepoly,locals={'z1':z1,'z2':z2})

In [2]:
#getting a list of monomials
mlist=buildmonoms(50) #you need to have at least as many monomials as optimal approximants, make sure you understand why
print(mlist)

[1, z1, z2, z1^2, z1*z2, z2^2, z1^3, z1^2*z2, z1*z2^2, z2^3, z1^4, z1^3*z2, z1^2*z2^2, z1*z2^3, z2^4, z1^5, z1^4*z2, z1^3*z2^2, z1^2*z2^3, z1*z2^4, z2^5, z1^6, z1^5*z2, z1^4*z2^2, z1^3*z2^3, z1^2*z2^4, z1*z2^5, z2^6, z1^7, z1^6*z2, z1^5*z2^2, z1^4*z2^3, z1^3*z2^4, z1^2*z2^5, z1*z2^6, z2^7, z1^8, z1^7*z2, z1^6*z2^2, z1^5*z2^3, z1^4*z2^4, z1^3*z2^5, z1^2*z2^6, z1*z2^7, z2^8, z1^9, z1^8*z2, z1^7*z2^2, z1^6*z2^3, z1^5*z2^4, z1^4*z2^5]


In [3]:
#Function zoo #fill in the other functions we discussed
p1=1-(z1+z2)/2
p2=4 - 3*z1 - z2 - z1*z2 +z1^2
p3=2*z1*z2 + (-1+i)*(z1+z2) - 2*i
p4=(z1+1)*p1

In [8]:
#Computing the OPAs and optimal norms
f=p1
howmany=15 #how many optimal approximants you need to compute
opa=[]
OptNorms=[]
for whichOA in range(0,howmany):
    JAMmat=buildMatrix(mlist,f,whichOA)
    #print(JAMmat)
    optimal_coeffs=list(JAMmat.solve_right(RHSmateq(f,whichOA)))
    #print optimal_coeffs
    Q.<z1,z2>=PolynomialRing(QQ,order='negdeglex')
    q=coeffpoly(optimal_coeffs,whichOA,mlist)
    opa=opa+[q]
    tonorm=q*f-1
    optimaldistsq=InnerProd(tonorm,tonorm,mlist)
    OptNorms=OptNorms+[optimaldistsq]

In [11]:
#displays the OPAs and optimal norms prettily
for counter in range(0,len(opa)):
    show(opa[counter])

2/3

3/4 + 1/4*z1

14/17 + 4/17*z1 + 4/17*z2

186/223 + 60/223*z1 + 52/223*z2 + 20/223*z1^2

1794/2039 + 684/2039*z1 + 620/2039*z2 + 160/2039*z1^2 + 408/2039*z1*z2

...

3459302/3755573 + 1434276/3755573*z1 + 1432484/3755573*z2 + 473040/3755573*z1^2 + 1086728/3755573*z1*z2 + 468560/3755573*z2^2 + 96432/3755573*z1^3 + 431776/3755573*z1^2*z2 + 432448/3755573*z1*z2^2 + 84112/3755573*z2^3 + 32144/3755573*z1^4

30662182/33136597 + 12989988/33136597*z1 + 12723364/33136597*z2 + 4755024/33136597*z1^2 + 9914440/33136597*z1*z2 + 4088464/33136597*z2^2 + 1544304/33136597*z1^3 + 4688000/33136597*z1^2*z2 + 3763712/33136597*z1*z2^2 + 735536/33136597*z2^3 + 173344/33136597*z1^4 + 2048544/33136597*z1^3*z2

6634066/7079471 + 2897900/7079471*z1 + 2845356/7079471*z2 + 1103600/7079471*z1^2 + 339816/1011353*z1*z2 + 972240/7079471*z2^2 + 295424/7079471*z1^3 + 1306832/7079471*z1^2*z2 + 1123968/7079471*z1*z2^2 + 19536/1011353*z2^3 + 30952/7079471*z1^4 + 405136/7079471*z1^3*z2 + 742744/7079471*z1^2*z2^2

588327882/625443187 + 257113916/625443187*z1 + 256983356/625443187*z2 + 96373968/625443187*z1^2 + 215131576/625443187*z1*z2 + 96047568/625443187*z2^2 + 25887120/625443187*z1^3 + 113686656/625443187*z1^2*z2 + 113821568/625443187*z1*z2^2 + 24903568/625443187*z2^3 + 2728704/625443187*z1^4 + 35402016/625443187*z1^3*z2 + 64006752/625443187*z1^2*z2^2 + 35573920/625443187*z1*z2^3

234530/249319 + 102476/249319*z1 + 102476/249319*z2 + 5488/35617*z1^2 + 85720/249319*z1*z2 + 5488/35617*z2^2 + 10320/249319*z1^3 + 45312/249319*z1^2*z2 + 45312/249319*z1*z2^2 + 10320/249319*z2^3 + 1088/249319*z1^4 + 2016/35617*z1^3*z2 + 25504/249319*z1^2*z2^2 + 2016/35617*z1*z2^3 + 1088/249319*z2^4

In [10]:
#displays the optimal norms prettily and then as a list that could be easily copy pasted
#for counter in range(0,len(OptNorms)):
    #show(OptNorms[counter]) #comment this out if you just want the copy-pastable version
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
for counter in range(0,len(OptNorms)):
    print(OptNorms[counter])

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1/3
1/4
3/17
37/223
245/2039
23/205
1549/13951
5867/61929
2739/34393
395/5001
296271/3755573
2474415/33136597
445405/7079471
37115305/625443187
14789/249319


In [10]:
#gives LaTeX output for the OPAs; you should be able to just copy paste this into Overleaf if you want
print("test function")
print("\\begin{equation*}")
print("\\scalebox{2}{\\ensuremath{f=",latex(f),"}}")
print("\\end{equation*}")
deg=0
print("\\begin{align*}")
print('p_{',deg,'}&=',latex(opa[0]),'\\\\')
for deg in range(1,len(opa)):
    print('p_{',deg,'}&=',latex(opa[deg]),'\\\\')
    print('%%%%%%%%%%%%%%%%%%%')
print("\\end{align*}")
print("")

test function
\begin{equation*}
\scalebox{2}{\ensuremath{f= 1 - \frac{1}{2} z_{1} - \frac{1}{2} z_{2} }}
\end{equation*}
\begin{align*}
p_{ 0 }&= \frac{2}{3} \\
p_{ 1 }&= \frac{3}{4} + \frac{1}{4} z_{1} \\
%%%%%%%%%%%%%%%%%%%
p_{ 2 }&= \frac{14}{17} + \frac{4}{17} z_{1} + \frac{4}{17} z_{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 3 }&= \frac{186}{223} + \frac{60}{223} z_{1} + \frac{52}{223} z_{2} + \frac{20}{223} z_{1}^{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 4 }&= \frac{1794}{2039} + \frac{684}{2039} z_{1} + \frac{620}{2039} z_{2} + \frac{160}{2039} z_{1}^{2} + \frac{408}{2039} z_{1} z_{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 5 }&= \frac{182}{205} + \frac{68}{205} z_{1} + \frac{68}{205} z_{2} + \frac{16}{205} z_{1}^{2} + \frac{8}{41} z_{1} z_{2} + \frac{16}{205} z_{2}^{2} \\
%%%%%%%%%%%%%%%%%%%
p_{ 6 }&= \frac{12402}{13951} + \frac{4684}{13951} z_{1} + \frac{660}{1993} z_{2} + \frac{1248}{13951} z_{1}^{2} + \frac{2712}{13951} z_{1} z_{2} + \frac{1088}{13951} z_{2}^{2} + \frac{416}{13951} z_{1}^{3} \\
%%%%%%%%%%%%%%%%

In [12]:
#LaTeX output for the optimal norms
print("\\begin{align*}")
for deg in range(0,len(OptNorms)):
    print('\\nu_{',deg,'}^2&=',latex(OptNorms[deg]),'\\\\')
    print('%%%%%%%%%%%%%%%%%%%')
print("\\end{align*}")

\begin{align*}
\nu_{ 0 }^2&= \frac{1}{2} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 1 }^2&= \frac{7}{15} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 2 }^2&= \frac{19}{51} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 3 }^2&= \frac{26}{85} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 4 }^2&= \frac{1304}{4297} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 5 }^2&= \frac{8119}{28391} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 6 }^2&= \frac{1295}{5039} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 7 }^2&= \frac{2267}{9419} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 8 }^2&= \frac{45083}{193133} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 9 }^2&= \frac{9370007}{40620743} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 10 }^2&= \frac{12970817}{62676400} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 11 }^2&= \frac{69395257}{336202393} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 12 }^2&= \frac{44771027}{225804307} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 13 }^2&= \frac{2807418853}{14476473773} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 14 }^2&= \frac{2300652517}{11882656247} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 15 }^2&= \frac{18962181745}{107475653897} \\
%%%%%%%%%%%%%%%%%%%
\nu_{ 16 }^2&= \frac{48324650941}{278942